In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import pickle

In [2]:
train_df_raw = pd.read_csv("./data/training.csv")

In [3]:
routes_df = pd.read_csv("./data/routes.csv")
route_list = routes_df['BUSROUTE_ID'].unique()

In [4]:
stop_df = pd.read_csv("./data/stops.csv")
stop_list = stop_df['BUSSTOP_ID'].unique()

In [5]:
bus_list = train_df_raw['BUS_ID'].unique()

In [6]:
train_df = train_df_raw

In [7]:
train_df.head()

,index,RECORD_DATE,BUS_ID,BUSROUTE_ID,BUSSTOP_ID,BUSSTOP_SEQ,TIMESTAMP
0,0,2020-01-06 07:00:00,325819008,11100360,388,21,1578294000
1,1,2020-01-06 07:00:00,451509115,11100480,112,9,1578294000
2,2,2020-01-06 07:00:00,235501055,11100500,374,9,1578294000
3,3,2020-01-06 07:00:00,145010001,11100330,331,30,1578294000
4,4,2020-01-06 07:00:01,421733004,11100620,51,29,1578294001


In [8]:
train_df = train_df.sort_values(by=['BUS_ID', 'TIMESTAMP'])

In [9]:
train_df['TIMESTAMP_DIFF'] = train_df['TIMESTAMP'].shift(-1) - train_df['TIMESTAMP']

In [10]:
train_df['SEQ_DIFF'] = train_df['BUSSTOP_SEQ'].shift(-1) - train_df['BUSSTOP_SEQ']

In [11]:
train_df['ROUTE_DIFF'] = train_df['BUSROUTE_ID'].shift(-1) - train_df['BUSROUTE_ID']

In [12]:
train_df['BUS_ID_DIFF'] = train_df['BUS_ID'].shift(-1) - train_df['BUS_ID']

In [13]:
# train_df = train_df[(train_df['SEQ_DIFF']==1.0) & (train_df['ROUTE_DIFF']==0.0) & (train_df['BUS_ID_DIFF']==0.0)]

In [14]:
train_df = train_df[train_df['SEQ_DIFF']==1]
train_df = train_df[train_df['ROUTE_DIFF']==0]
train_df = train_df[train_df['BUS_ID_DIFF']==0]

In [15]:
train_df[train_df['TIMESTAMP_DIFF']<1]

,index,RECORD_DATE,BUS_ID,BUSROUTE_ID,BUSSTOP_ID,BUSSTOP_SEQ,TIMESTAMP,TIMESTAMP_DIFF,SEQ_DIFF,ROUTE_DIFF,BUS_ID_DIFF


In [16]:
# bus id encoding
bus_id_encoder = preprocessing.LabelEncoder()
bus_id_encoder.fit(bus_list)
train_df.BUS_ID = bus_id_encoder.transform(train_df.BUS_ID)

# bus_id_encoder.transform([145010001, 235501055])
# bus_id_encoder.inverse_transform([280, 466])

In [17]:
# route id encoding
route_id_encoder = preprocessing.LabelEncoder()
route_id_encoder.fit(route_list)
train_df.BUSROUTE_ID = route_id_encoder.transform(train_df.BUSROUTE_ID)

In [18]:
# stop id encoding
stop_id_encoder = preprocessing.LabelEncoder()
stop_id_encoder.fit(stop_list)
train_df.BUSSTOP_ID = stop_id_encoder.transform(train_df.BUSSTOP_ID)

In [19]:
pickle.dump(bus_id_encoder  , open("./out/bus_id_encoder.pickle"    , "wb"))
pickle.dump(route_id_encoder, open("./out/route_id_encoder.pickle"  , "wb"))
pickle.dump(stop_id_encoder , open("./out/busstop_id_encoder.pickle", "wb"))

In [20]:
train_df['DAY_OF_WEEK'] = pd.to_datetime(train_df['RECORD_DATE']).dt.dayofweek

In [21]:
def timestamp_to_timeposition(time, starting_time = 21600): 
    if time < starting_time:
        return 0
    return (time - starting_time) % 86400

In [22]:
def timestamp_to_dayofweek(time):
    return (time//3600//24 + 3) % 7

In [23]:
timestamp_to_timeposition(1578607757)

58157

In [24]:
# time validation (valid time is 06:00:00 ~ 23:59:59)
starting_time = 21600 # 06:00:00
train_df['TIME_validation'] = train_df['TIMESTAMP'] % 86400
train_df = train_df[train_df['TIME_validation']>=starting_time]

In [25]:
# encodeing time

# (pd.to_datetime(train_df['RECORD_DATE']) - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
# (pd.Timestamp('2020-01-14 07:00:00')  - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
train_df['TIME'] = (train_df['TIMESTAMP'] - starting_time) % 86400


In [26]:
train_df.sort_values(by=['TIMESTAMP_DIFF'])

,index,RECORD_DATE,BUS_ID,BUSROUTE_ID,BUSSTOP_ID,BUSSTOP_SEQ,TIMESTAMP,TIMESTAMP_DIFF,SEQ_DIFF,ROUTE_DIFF,BUS_ID_DIFF,DAY_OF_WEEK,TIME_validation,TIME
2495182,2495182,2020-01-16 13:20:24,610,206,378,16,1579180824,3.0,1.0,0.0,0.0,3,48024,26424
257759,257759,2020-01-07 07:23:00,296,260,1170,47,1578381780,4.0,1.0,0.0,0.0,1,26580,4980
80314,80314,2020-01-06 11:18:34,725,251,379,27,1578309514,4.0,1.0,0.0,0.0,0,40714,19114
1170937,1170937,2020-01-10 14:41:21,427,41,452,40,1578667281,5.0,1.0,0.0,0.0,4,52881,31281
2543779,2543779,2020-01-16 16:05:54,200,251,379,27,1579190754,5.0,1.0,0.0,0.0,3,57954,36354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1274219,1274219,2020-01-10 21:28:38,710,135,163,8,1578691718,207631.0,1.0,0.0,0.0,4,77318,55718
1031576,1031576,2020-01-09 22:09:17,468,200,1158,21,1578607757,208388.0,1.0,0.0,0.0,3,79757,58157
1238330,1238330,2020-01-10 19:06:14,219,265,148,13,1578683174,214548.0,1.0,0.0,0.0,4,68774,47174
1162737,1162737,2020-01-10 14:12:16,800,223,1194,25,1578665536,232925.0,1.0,0.0,0.0,4,51136,29536


In [27]:
# 2000 sec-ees ihiig evdersen esvel tur zogsolt hiisen gej uzeed haslaa

# train_df[(train_df['TIMESTAMP_DIFF']>2000)].sort_values(by=['TIMESTAMP_DIFF'])
train_df = train_df[(train_df['TIMESTAMP_DIFF']<2000)]

In [28]:
output_df = train_df[["DAY_OF_WEEK", "TIME", "BUS_ID", "BUSROUTE_ID", "BUSSTOP_ID", "TIMESTAMP_DIFF"]]

In [29]:
output_df.to_csv('./out/cleaned_train.csv', index = False)


In [30]:
train_df.head(10000)

,index,RECORD_DATE,BUS_ID,BUSROUTE_ID,BUSSTOP_ID,BUSSTOP_SEQ,TIMESTAMP,TIMESTAMP_DIFF,SEQ_DIFF,ROUTE_DIFF,BUS_ID_DIFF,DAY_OF_WEEK,TIME_validation,TIME
1624542,1624542,2020-01-13 07:28:25,0,3,17,2,1578900505,117.0,1.0,0.0,0.0,0,26905,5305
1625151,1625151,2020-01-13 07:30:22,0,3,10,3,1578900622,123.0,1.0,0.0,0.0,0,27022,5422
1625791,1625791,2020-01-13 07:32:25,0,3,8,4,1578900745,121.0,1.0,0.0,0.0,0,27145,5545
1626425,1626425,2020-01-13 07:34:26,0,3,5,5,1578900866,136.0,1.0,0.0,0.0,0,27266,5666
1627161,1627161,2020-01-13 07:36:42,0,3,3,6,1578901002,174.0,1.0,0.0,0.0,0,27402,5802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2354147,2354147,2020-01-15 20:38:59,6,16,283,20,1579120739,282.0,1.0,0.0,0.0,2,74339,52739
2355528,2355528,2020-01-15 20:43:41,6,16,311,21,1579121021,297.0,1.0,0.0,0.0,2,74621,53021
2356883,2356883,2020-01-15 20:48:38,6,16,1191,22,1579121318,380.0,1.0,0.0,0.0,2,74918,53318
2358597,2358597,2020-01-15 20:54:58,6,16,407,23,1579121698,99.0,1.0,0.0,0.0,2,75298,53698


In [31]:
#diff route num = 102
df = train_df.head(10000)
df = df.sort_values(by=['BUSROUTE_ID','BUS_ID', 'TIMESTAMP'])

In [32]:
route_index = 0
prev_seq = 0
seq_data = []
for idx, row in df.iterrows():
    if prev_seq == 0:
        prev_bus = row['BUS_ID']
        prev_seq = row['BUSSTOP_SEQ']
        prev_route = row['BUSROUTE_ID']
        prev_timestamp = row['TIMESTAMP']
        min_timestamp = row['TIMESTAMP']
        cnt = 1
        continue
        
    if (row['BUSSTOP_SEQ']>prev_seq and row['BUS_ID'] == prev_bus and row['BUSROUTE_ID'] == prev_route):
        pass
    else:
        seq_data.append([route_index, prev_route, cnt, (prev_timestamp - min_timestamp)/3600])
        min_timestamp = row['TIMESTAMP']
        route_index+=1
        cnt = 0

        
        
    prev_bus = row['BUS_ID']
    prev_seq = row['BUSSTOP_SEQ']
    prev_route = row['BUSROUTE_ID']
    prev_timestamp = row['TIMESTAMP']
    df.at[idx, 'ROUTE_IDX'] = route_index
    cnt += 1
    
seq_data.append([route_index, prev_route, cnt, (prev_timestamp - min_timestamp)/3600])

#   if (pd.isna(row['TIMESTAMP'])):
#     timestamp = test_df.iloc[idx-1]['TIMESTAMP']
#     prev_row  = test_df.iloc[idx-1]
#     if (not pd.isna(timestamp)):
#       day_of_week   = int(timestamp_to_dayofweek(timestamp))
#       time_position = int(timestamp_to_timeposition(timestamp))
      
#       bus_id = int(prev_row['BUS_ID'])
#       try:
#         bus_position = bus_id_encoder.transform([bus_id])[0]
#       except:
#         bus_position = random.choice(cleaned_buses)
      
#       route_id       = int(prev_row['BUSROUTE_ID'])
#       route_position = route_id_encoder.transform([route_id])[0]
      
#       busstop_id       = int(prev_row['BUSSTOP_ID'])
#       busstop_position = busstop_id_encoder.transform([busstop_id])[0]
#       prediction = c_model.predict([day_of_week, time_position, bus_position, route_position, busstop_position])
      
#       # model_input = np.array([[day_of_week, time_position, bus_position, route_position, busstop_position]])
#       # prediction  = int(model(model_input, training=False))

      
#       test_df.at[idx, 'TIMESTAMP'] = timestamp + prediction

In [33]:
df

,index,RECORD_DATE,BUS_ID,BUSROUTE_ID,BUSSTOP_ID,BUSSTOP_SEQ,TIMESTAMP,TIMESTAMP_DIFF,SEQ_DIFF,ROUTE_DIFF,BUS_ID_DIFF,DAY_OF_WEEK,TIME_validation,TIME,ROUTE_IDX
1624542,1624542,2020-01-13 07:28:25,0,3,17,2,1578900505,117.0,1.0,0.0,0.0,0,26905,5305,NaN
1625151,1625151,2020-01-13 07:30:22,0,3,10,3,1578900622,123.0,1.0,0.0,0.0,0,27022,5422,0.0
1625791,1625791,2020-01-13 07:32:25,0,3,8,4,1578900745,121.0,1.0,0.0,0.0,0,27145,5545,0.0
1626425,1626425,2020-01-13 07:34:26,0,3,5,5,1578900866,136.0,1.0,0.0,0.0,0,27266,5666,0.0
1627161,1627161,2020-01-13 07:36:42,0,3,3,6,1578901002,174.0,1.0,0.0,0.0,0,27402,5802,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3104592,3104592,2020-01-19 11:41:57,5,214,1119,20,1579434117,159.0,1.0,0.0,0.0,6,42117,20517,300.0
3105096,3105096,2020-01-19 11:44:36,5,214,540,21,1579434276,237.0,1.0,0.0,0.0,6,42276,20676,300.0
3105808,3105808,2020-01-19 11:48:33,5,214,538,22,1579434513,146.0,1.0,0.0,0.0,6,42513,20913,300.0
3106283,3106283,2020-01-19 11:50:59,5,214,534,23,1579434659,98.0,1.0,0.0,0.0,6,42659,21059,300.0


In [34]:
seq_data

[[0, 3, 20, 0.7730555555555556],
 [1, 3, 9, 0.4047222222222222],
 [2, 3, 3, 0.1275],
 [3, 3, 35, 1.9547222222222222],
 [4, 3, 19, 0.6913888888888889],
 [5, 3, 27, 1.3133333333333332],
 [6, 3, 16, 0.9452777777777778],
 [7, 3, 23, 1.2166666666666666],
 [8, 3, 19, 48.58555555555556],
 [9, 3, 38, 1.7855555555555556],
 [10, 3, 38, 1.8133333333333332],
 [11, 3, 38, 1.635],
 [12, 3, 38, 1.9505555555555556],
 [13, 3, 38, 2.4530555555555558],
 [14, 3, 38, 2.4830555555555556],
 [15, 3, 38, 1.3766666666666667],
 [16, 3, 38, 1.57],
 [17, 3, 38, 1.6919444444444445],
 [18, 3, 38, 2.0477777777777777],
 [19, 3, 38, 1.9766666666666666],
 [20, 3, 38, 1.7269444444444444],
 [21, 3, 38, 1.7497222222222222],
 [22, 3, 36, 1.8658333333333332],
 [23, 3, 19, 1.1055555555555556],
 [24, 3, 38, 1.5030555555555556],
 [25, 3, 38, 1.5663888888888888],
 [26, 3, 38, 1.8780555555555556],
 [27, 3, 38, 1.9469444444444444],
 [28, 3, 19, 0.9552777777777778],
 [29, 3, 38, 1.7277777777777779],
 [30, 3, 38, 2.082222222222222],

In [35]:
df=df.fillna(0)

In [36]:
for idx, row in df.iterrows():
    df.at[idx, 'ROUTE_TIME'] = seq_data[int(row['ROUTE_IDX'])][3]
    df.at[idx, 'SEQ_NUM'] = seq_data[int(row['ROUTE_IDX'])][2]

In [37]:
# 1578900505, 1578992933
# [[0, 1579118733, 1579119798],
#  [1, 1578900505, 1578903288],
#  [2, 1578991476, 1578992933],
# seq_data
df.head(60)

,index,RECORD_DATE,BUS_ID,BUSROUTE_ID,BUSSTOP_ID,BUSSTOP_SEQ,TIMESTAMP,TIMESTAMP_DIFF,SEQ_DIFF,ROUTE_DIFF,BUS_ID_DIFF,DAY_OF_WEEK,TIME_validation,TIME,ROUTE_IDX,ROUTE_TIME,SEQ_NUM
1624542,1624542,2020-01-13 07:28:25,0,3,17,2,1578900505,117.0,1.0,0.0,0.0,0,26905,5305,0.0,0.773056,20.0
1625151,1625151,2020-01-13 07:30:22,0,3,10,3,1578900622,123.0,1.0,0.0,0.0,0,27022,5422,0.0,0.773056,20.0
1625791,1625791,2020-01-13 07:32:25,0,3,8,4,1578900745,121.0,1.0,0.0,0.0,0,27145,5545,0.0,0.773056,20.0
1626425,1626425,2020-01-13 07:34:26,0,3,5,5,1578900866,136.0,1.0,0.0,0.0,0,27266,5666,0.0,0.773056,20.0
1627161,1627161,2020-01-13 07:36:42,0,3,3,6,1578901002,174.0,1.0,0.0,0.0,0,27402,5802,0.0,0.773056,20.0
1628090,1628090,2020-01-13 07:39:36,0,3,146,7,1578901176,106.0,1.0,0.0,0.0,0,27576,5976,0.0,0.773056,20.0
1628653,1628653,2020-01-13 07:41:22,0,3,148,8,1578901282,224.0,1.0,0.0,0.0,0,27682,6082,0.0,0.773056,20.0
1629874,1629874,2020-01-13 07:45:06,0,3,150,9,1578901506,144.0,1.0,0.0,0.0,0,27906,6306,0.0,0.773056,20.0
1630675,1630675,2020-01-13 07:47:30,0,3,1011,10,1578901650,199.0,1.0,0.0,0.0,0,28050,6450,0.0,0.773056,20.0
1631768,1631768,2020-01-13 07:50:49,0,3,273,11,1578901849,119.0,1.0,0.0,0.0,0,28249,6649,0.0,0.773056,20.0


In [38]:
train_df.groupby('BUSROUTE_ID').agg({'BUSROUTE_ID':'size', 'TIMESTAMP_DIFF':'mean'}) \
.rename(columns={'BUSROUTE_ID':'count','TIMESTAMP_DIFF':'mean_sent'}).sort_values(by=['mean_sent'])

,count,mean_sent
BUSROUTE_ID,,
246,1,50.000000
107,2,78.000000
201,2,90.500000
41,42076,91.633354
266,6,92.166667
...,...,...
162,28071,270.877347
270,4231,280.531553
193,1244,281.928457


In [39]:
routes_df.groupby(['BUSROUTE_ID']).agg([('Min' , 'min'), ('Max', 'max'), ('SIZE', 'size')])
#        .add_prefix('Day'))

BUSSTOP_ID                 BUSSTOP_SEQ         
                   Min        Max SIZE         Min Max SIZE
BUSROUTE_ID                                                
1110049            373  100000005   24           1  24   24
1110059              4  100000010   50           1  50   50
1110130             36        848    4           1   4    4
11100010             4  100000010   40           1  40   40
11100012             4       1412   38           1  38   38
...                ...        ...  ...         ...  ..  ...
12100210           360       1534    9           1   9    9
12100220           115       1245    9           1   9    9
12100230           192        657   10           1  10   10
12100231           192        657    6           1   6    6
12100232           192        657   10           1  10   10

[353 rows x 6 columns]

In [40]:
# one_bus  = train_df[train_df['BUS_ID']==800]

In [41]:
# one_bus.to_excel("output.xlsx")

In [42]:
# one_bus = one_bus.dropna()